In [130]:
#What is the probability that I will get booked if I charge $35/night with 4 repeat guests and 10 reviews?

In [28]:
import pandas as pd
import statsmodels.api as sm
import math
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
from os.path import join
from sklearn import metrics

In [29]:
df = pd.read_csv('dog-vacay.csv')

In [30]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [31]:
df['fee'].mean()

38.678464178464175

In [32]:
df['repeat'].mean()

4.8178893178893176

In [33]:
df['review'].mean()

9.8065208065208065

In [34]:
df['fee35'] = df['fee'].map(lambda x: 1 if x >= 35 else 0)

In [35]:
cols = ['repeat', 'review']

In [36]:
x = df[cols]
y = df['fee35']

In [37]:
#Define the logistic regression model.
model = sm.Logit(y,x)

In [38]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.687719
         Iterations 5


In [39]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  fee35   No. Observations:                 4662
Model:                          Logit   Df Residuals:                     4660
Method:                           MLE   Df Model:                            1
Date:                Tue, 05 Jan 2016   Pseudo R-squ.:                -0.07994
Time:                        20:59:44   Log-Likelihood:                -3206.1
converged:                       True   LL-Null:                       -2968.8
                                        LLR p-value:                     1.000
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
repeat        -0.0118      0.009     -1.314      0.189        -0.029     0.006
review         0.0158      0.005      3.378      0.001         0.007     0.025
==============================================================================
"""

In [40]:
def logistic_function(repeat, review):
    prob = 1/(1 + math.exp(coeff[0]*repeat+coeff[1]*review))
    if prob > 0.7:
        p = 1
    else:
        p = 0
    return prob, p

In [41]:
coeff = result.params
print coeff

repeat   -0.011773
review    0.015812
dtype: float64


In [42]:
logistic_function(4, 9)

(0.47621438214851924, 0)

In [43]:
#SVC TEST

In [44]:
dfTrain, dfTest = train_test_split(df, test_size=0.1, random_state=0)

In [45]:
dfTrain = pd.DataFrame(dfTrain, columns=df.columns)
dfTest = pd.DataFrame(dfTest, columns=df.columns)

In [46]:
x1 = dfTrain[cols]
y1 = dfTrain['fee35']

In [47]:
#Train
clf = SVC(gamma=.001)

In [48]:
clf.fit(x1,y1)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

In [49]:
#Test
x2 = dfTest[cols]
y2 = dfTest['fee35']

In [50]:
ypred = clf.predict(x2)

In [70]:
#Converts the array to one with data type np.float64 (i.e. an array where the array elements 
#are the actual floating point values, not pointers to objects)
a = ypred.astype(np.float64)

In [71]:
fpr, tpr, thresholds = metrics.roc_curve(y2, a, pos_label=1)
AUC = metrics.auc(fpr, tpr)

In [74]:
AUC

0.49980773122849625